Paige Haring

peh40@pitt.edu

10/16/17

This notebook is my progress following along with this tutorial on scikit-learn.org: http://scikit-learn.org/dev/tutorial/text_analytics/working_with_text_data.html

In [3]:
import sklearn
import nltk
from sklearn.datasets import load_files

In [7]:
#setting directory to where my nltk data has the movie reviews corpus
nltk.data.path #gotta find it first...
movie_dir = "/Users/Paige/nltk_data/corpora/movie_reviews/"

#load all of the files in the movie_reviews corpus as training data
movie_train = load_files(movie_dir, shuffle=True)

In [9]:
#What is movie_train?
type(movie_train)

sklearn.datasets.base.Bunch

In [10]:
dir(movie_train)

['DESCR', 'data', 'filenames', 'target', 'target_names']

In [11]:
movie_train.target_names #These are the labels or classes we will want to predict

['neg', 'pos']

In [12]:
movie_train.target

array([0, 1, 1, ..., 1, 0, 0])

In [13]:
movie_train.filenames

array(['/Users/Paige/nltk_data/corpora/movie_reviews/neg/cv405_21868.txt',
       '/Users/Paige/nltk_data/corpora/movie_reviews/pos/cv190_27052.txt',
       '/Users/Paige/nltk_data/corpora/movie_reviews/pos/cv132_5618.txt',
       ...,
       '/Users/Paige/nltk_data/corpora/movie_reviews/pos/cv653_19583.txt',
       '/Users/Paige/nltk_data/corpora/movie_reviews/neg/cv559_0057.txt',
       '/Users/Paige/nltk_data/corpora/movie_reviews/neg/cv684_12727.txt'], 
      dtype='<U64')

In [16]:
#How many reviews do we have?
len(movie_train.data)

2000